# 多元统计分析作业——爬取安居客房价

> 所有数据均来源于安居客网站。    
> Copyright © 2007-2019 www.anjuke.com All Rights Reserved

In [2]:
# Requirements: requests, numpy, pandas, lxml, time, csv, sys
# 必要时可以设置time.sleep(2)消除屏蔽机制，不过安居客好像没有

url = 'https://www.anjuke.com/fangjia/'
HEADERS={
    'Cookie': '',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,zh-TW;q=0.6',
}

# 爬取省的链接以及名称

In [2]:
import requests
from lxml import etree

response = requests.get(url,headers=HEADERS)
text = response.text
main_page = etree.HTML(text)
province_links = main_page.xpath('/html/body/div[2]/div[2]/div/span[2]/a/@href')
province = main_page.xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()')

In [3]:
print(province_links)
print(province)

['http://www.anjuke.com/fangjia/yixianchengshi/', 'http://www.anjuke.com/fangjia/anhui/', 'http://www.anjuke.com/fangjia/fujian/', 'http://www.anjuke.com/fangjia/gansu/', 'http://www.anjuke.com/fangjia/guangdong/', 'http://www.anjuke.com/fangjia/guangxi/', 'http://www.anjuke.com/fangjia/guizhou/', 'http://www.anjuke.com/fangjia/hainan/', 'http://www.anjuke.com/fangjia/hebei/', 'http://www.anjuke.com/fangjia/heilongjiang/', 'http://www.anjuke.com/fangjia/henan/', 'http://www.anjuke.com/fangjia/hubei/', 'http://www.anjuke.com/fangjia/hunan/', 'http://www.anjuke.com/fangjia/jiangsu/', 'http://www.anjuke.com/fangjia/jiangxi/', 'http://www.anjuke.com/fangjia/jilin/', 'http://www.anjuke.com/fangjia/liaoning/', 'http://www.anjuke.com/fangjia/neimenggu/', 'http://www.anjuke.com/fangjia/ningxia/', 'http://www.anjuke.com/fangjia/qinghai/', 'http://www.anjuke.com/fangjia/shan3xi/', 'http://www.anjuke.com/fangjia/shandong/', 'http://www.anjuke.com/fangjia/shanxi/', 'http://www.anjuke.com/fangjia/s

# 爬取市的链接和名称

In [4]:
# 收集城市
city_links_all = []
city_all = []
for province_url in province_links:
    province_response = requests.get(province_url,headers=HEADERS)
    city_links_all.append(etree.HTML(province_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/div/a/@href'))
    city_all.append(etree.HTML(province_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/div/a/text()'))

In [5]:
print(city_all) 
# 可以看到城市之中有重复的，所以需要进行清洗去除重复的城市;而且这里还存在空的情况，海南和吉林后面会进行处理

[['北京', '重庆', '上海', '天津'], ['安庆', '蚌埠', '亳州', '巢湖', '池州', '滁州', '阜阳', '合肥', '和县', '霍邱', '淮北', '淮南', '黄山', '霍邱', '六安', '马鞍山', '宿州', '桐城', '桐城', '铜陵', '芜湖', '宣城'], ['福州', '龙岩', '南平', '宁德', '莆田', '泉州', '三明', '武夷山', '厦门', '漳州'], ['白银', '定西', '定西', '甘南', '甘南', '嘉峪关', '金昌', '金昌', '酒泉', '嘉峪关', '兰州', '临夏', '陇南', '陇南', '临夏', '平凉', '平凉', '庆阳', '天水', '武威', '张掖', '张掖'], ['潮州', '东莞', '佛山', '广州', '河源', '惠州', '江门', '揭阳', '茂名', '梅州', '清远', '汕头', '汕尾', '韶关', '深圳', '顺德', '台山', '台山', '阳春', '阳春', '阳江', '云浮', '云浮', '湛江', '肇庆', '中山', '珠海'], ['百色', '北海', '崇左', '崇左', '防城港', '防城港', '贵港', '桂林', '河池', '河池', '贺州', '贺州', '来宾', '来宾', '柳州', '南宁', '钦州', '梧州', '玉林'], ['安顺', '毕节', '毕节', '贵阳', '六盘水', '黔东南', '黔东南', '黔南', '黔西南', '黔南', '黔西南', '铜仁', '铜仁', '遵义'], [], ['保定', '沧州', '承德', '定州', '定州', '馆陶', '邯郸', '衡水', '廊坊', '秦皇岛', '石家庄', '唐山', '邢台', '张北', '张家口', '赵县', '正定', '赵县'], ['大庆', '大兴安岭', '大兴安岭', '哈尔滨', '鹤岗', '黑河', '黑河', '佳木斯', '鸡西', '牡丹江', '齐齐哈尔', '七台河', '七台河', '双鸭山', '绥化', '双鸭山', '伊春', '伊春'], ['安阳', '长葛', '长葛', '鹤壁', '

# 清洗数据去除重复值

In [6]:
# 去重函数
def remove_repeat_city(links):
    temp = []
    for i in links:
        newList = []
        for j in i:
            if j not in newList:
                newList.append(j)
        temp.append(newList)
    return temp

# 验证链接和名称是否对应

In [7]:
city_links = []
city = []

city_links = remove_repeat_city(city_links_all)
city = remove_repeat_city(city_all)

# 检查二者长度是否相等
print(len(city_all) == len(city_links_all))
for i in range(len(city_links_all)):
    if len(city_all[i]) != len(city_links_all[i]):
        print("Error!")

True


# 处理异常数据

In [10]:
# 城市中有为空的情况
# 对于海南，安居客把他放在了青海,不懂什么操作，情况比较复杂，一杯咖啡后决定抛弃海南，对不起海南；
# 对于吉林，列表里只有吉林市的数据而不是吉林省的，单独写一个加进去就行
print(city[7])
print(city[15])
print(city_links[7])
print(city_links[15])

[]
[]
[]
[]


In [14]:
city.pop(7)
city_links.pop(7)
city_links[14] = ['https://www.anjuke.com/fangjia/jilin/']
city[14] = ['吉林']

In [15]:
print(city)

[['北京', '重庆', '上海', '天津'], ['安庆', '蚌埠', '亳州', '巢湖', '池州', '滁州', '阜阳', '合肥', '和县', '霍邱', '淮北', '淮南', '黄山', '六安', '马鞍山', '宿州', '桐城', '铜陵', '芜湖', '宣城'], ['福州', '龙岩', '南平', '宁德', '莆田', '泉州', '三明', '武夷山', '厦门', '漳州'], ['白银', '定西', '甘南', '嘉峪关', '金昌', '酒泉', '兰州', '临夏', '陇南', '平凉', '庆阳', '天水', '武威', '张掖'], ['潮州', '东莞', '佛山', '广州', '河源', '惠州', '江门', '揭阳', '茂名', '梅州', '清远', '汕头', '汕尾', '韶关', '深圳', '顺德', '台山', '阳春', '阳江', '云浮', '湛江', '肇庆', '中山', '珠海'], ['百色', '北海', '崇左', '防城港', '贵港', '桂林', '河池', '贺州', '来宾', '柳州', '南宁', '钦州', '梧州', '玉林'], ['安顺', '毕节', '贵阳', '六盘水', '黔东南', '黔南', '黔西南', '铜仁', '遵义'], ['保定', '沧州', '承德', '定州', '馆陶', '邯郸', '衡水', '廊坊', '秦皇岛', '石家庄', '唐山', '邢台', '张北', '张家口', '赵县', '正定'], ['大庆', '大兴安岭', '哈尔滨', '鹤岗', '黑河', '佳木斯', '鸡西', '牡丹江', '齐齐哈尔', '七台河', '双鸭山', '绥化', '伊春'], ['安阳', '长葛', '鹤壁', '焦作', '济源', '开封', '漯河', '洛阳', '明港', '南阳', '平顶山', '濮阳', '三门峡', '商丘', '新乡', '信阳', '许昌', '鄢陵', '禹州', '郑州', '周口', '驻马店'], ['恩施', '鄂州', '黄冈', '黄石', '荆门', '荆州', '潜江', '神农架', '十堰', '随州', '天门', '武汉', '襄阳', '

In [16]:
# 再来检查一遍
for i in range(len(city_links)):
    if len(city[i]) != len(city_links[i]):
        print("Error!")

In [17]:
# 同样的，省里面得去掉海南，再次向海南道歉
province.pop(7)
province_links.pop(7)

'http://www.anjuke.com/fangjia/hainan/'

In [18]:
# 有多少城市呢？
temp = 0
for i in range(len(city)):
    for j in range(len(city[i])):
        temp += 1
print(temp)

366


# 收集县区名称以及链接

In [23]:
# 收集县区
import time
district_links = []
district = []
count_p = 0
for i in range(len(city_links)):
    temp = []
    temp_links = []
    for j in range(len(city_links[i])):
        district_response = requests.get(city_links[i][j],headers=HEADERS)
        if etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()') != []:
            if etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()')[0] != '全部':
                temp.append(etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()'))
                temp_links.append(etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/@href'))
            else:
                #应对地级市的情况
                temp.append(city[i][j])
                temp_links.append(city_links[i][j])
#         time.sleep(0.5)
        print('get %s' % count_p)
        count_p += 1
    district.append(temp)
    district_links.append(temp_links)

get 0
get 1
get 2
get 3
get 4
get 5
get 6
get 7
get 8
get 9
get 10
get 11
get 12
get 13
get 14
get 15
get 16
get 17
get 18
get 19
get 20
get 21
get 22
get 23
get 24
get 25
get 26
get 27
get 28
get 29
get 30
get 31
get 32
get 33
get 34
get 35
get 36
get 37
get 38
get 39
get 40
get 41
get 42
get 43
get 44
get 45
get 46
get 47
get 48
get 49
get 50
get 51
get 52
get 53
get 54
get 55
get 56
get 57
get 58
get 59
get 60
get 61
get 62
get 63
get 64
get 65
get 66
get 67
get 68
get 69
get 70
get 71
get 72
get 73
get 74
get 75
get 76
get 77
get 78
get 79
get 80
get 81
get 82
get 83
get 84
get 85
get 86
get 87
get 88
get 89
get 90
get 91
get 92
get 93
get 94
get 95
get 96
get 97
get 98
get 99
get 100
get 101
get 102
get 103
get 104
get 105
get 106
get 107
get 108
get 109
get 110
get 111
get 112
get 113
get 114
get 115
get 116
get 117
get 118
get 119
get 120
get 121
get 122
get 123
get 124
get 125
get 126
get 127
get 128
get 129
get 130
get 131
get 132
get 133
get 134
get 135
get 136
get 137
get 13

# 验证数据并处理错误数据

In [24]:
# 验证数据长度,有些市没有列出县区所以会出错，比如安庆，合肥，但是我昨晚试的时候安庆还是被列出来的，可能服务器出于维护期
for i in range(len(district)):
    if len(district[i]) != len(district_links[i]):
            print('error!',i)
    for j in range(len(district[i])):
        if len(district[i][j]) != len(district_links[i][j]):
            print('error!',i,j)

error! 1 0
error! 1 7
error! 1 13
error! 3 8
error! 3 13
error! 5 0
error! 5 2
error! 5 5
error! 6 0
error! 7 2
error! 7 5
error! 9 5
error! 12 0
error! 12 8
error! 12 10
error! 13 4
error! 15 12
error! 19 3
error! 21 9
error! 22 8
error! 26 2


In [28]:
for i in range(len(district)):
    for j in range(len(district[i])):
        if len(district[i][j]) != len(district_links[i][j]):
            print(district[i][j])
            print(district_links[i][j])

安庆
http://www.anjuke.com/fangjia/anqing/
合肥
http://www.anjuke.com/fangjia/hf/
六安
http://www.anjuke.com/fangjia/luan/
陇南
http://www.anjuke.com/fangjia/longnan/
张掖
http://www.anjuke.com/fangjia/zhangye/
百色
http://www.anjuke.com/fangjia/baise/
崇左
http://www.anjuke.com/fangjia/chongzuo/
桂林
http://www.anjuke.com/fangjia/guilin/
安顺
http://www.anjuke.com/fangjia/anshun/
承德
http://www.anjuke.com/fangjia/chengde/
邯郸
http://www.anjuke.com/fangjia/handan/
开封
http://www.anjuke.com/fangjia/kaifeng/
常州
http://www.anjuke.com/fangjia/cz/
宿迁
http://www.anjuke.com/fangjia/suqian/
太仓
http://www.anjuke.com/fangjia/taicang/
九江
http://www.anjuke.com/fangjia/jiujiang/
铁岭
http://www.anjuke.com/fangjia/tieling/
商洛
http://www.anjuke.com/fangjia/shangluo/
太原
http://www.anjuke.com/fangjia/ty/
乐山
http://www.anjuke.com/fangjia/leshan/
嘉兴
http://www.anjuke.com/fangjia/jx/


In [30]:
print(district)

[[['朝阳', '海淀', '东城', '西城', '丰台', '通州', '石景山', '昌平', '大兴', '顺义', '房山', '门头沟', '密云', '怀柔', '平谷', '延庆', '北京周边'], ['渝北', '江北', '沙坪坝', '南岸', '九龙坡', '渝中', '巴南', '大渡口', '北碚', '万州', '璧山', '合川', '永川', '江津', '涪陵', '铜梁', '长寿', '潼南', '荣昌', '开州', '大足', '南川', '垫江', '綦江', '万盛', '梁平', '丰都', '武隆', '奉节', '云阳', '石柱', '秀山', '忠县', '彭水', '黔江', '巫山', '酉阳', '城口', '巫溪', '其它'], ['浦东', '闵行', '宝山', '徐汇', '松江', '嘉定', '静安', '普陀', '杨浦', '虹口', '长宁', '黄浦', '青浦', '奉贤', '金山', '崇明', '上海周边'], ['和平', '河西', '南开', '河北', '河东', '红桥', '滨海新区', '开发区', '西青', '津南', '东丽', '北辰', '大港', '武清', '宝坻', '蓟州', '静海', '宁河', '汉沽', '塘沽', '天津周边']], ['安庆', ['蚌山', '龙子湖', '禹会', '淮上', '怀远', '固镇', '五河', '其他'], ['利辛', '蒙城', '涡阳', '谯城'], ['居巢', '庐江', '含山', '和县', '巢湖周边'], ['贵池', '东至', '石台', '青阳', '池州周边'], ['琅琊', '南谯', '来安', '全椒', '定远', '凤阳', '天长', '明光', '滁州周边'], ['颍州', '颍泉', '颍东', '经济开发区', '界首', '阜南', '太和', '颍上', '临泉'], '合肥', ['历阳镇', '沈巷镇', '白桥镇', '姥桥镇', '功桥镇', '西埠镇', '香泉镇', '乌江镇', '善厚镇', '石杨镇'], ['城关', '河口', '周集', '姚李', '众兴', '新店', '长集', '马店', '扈胡', '霍邱

In [31]:
# 上面出错的原因是插入错误，我们期望得到的输出应该是  ['安庆' ] 这样子
for i in range(len(district)):
    for j in range(len(district[i])):
        if len(district[i][j]) != len(district_links[i][j]):
            temp = []
            temp_link = []
            temp.append(district[i][j])
            temp_link.append(district_links[i][j])
            district[i][j] = []
            district_links[i][j] = []
            district[i][j].append(temp)
            district_links[i][j].append(temp_link)

In [33]:
# 看看修补完成没
for i in range(len(district)):
    for j in range(len(district[i])):
        if len(district[i][j]) != len(district_links[i][j]):
            print(district[i][j])
            print(district_links[i][j])

for i in range(len(district)):
    if len(district[i]) != len(district_links[i]):
            print('error!',i)
    for j in range(len(district[i])):
        if len(district[i][j]) != len(district_links[i][j]):
            print('error!',i,j)

# 用于格式化输入CSV

In [5]:
# 现在已经获得了县区，市，省的地址和名称，接下来一页页爬取数据就行了，爬到的数据需要格式化一下输入表格中，所以写几个函数来格式化数据

# 返回房价数据
def make_fj(lists):
    temp = []
    for list_ in lists:
        temp.append(list_[:-3])
    return temp

# 返回涨幅数据
def make_fd(lists):
    temp = []
    for list_ in lists:
        if list_[-1] == '↓':
            temp.append('-' + list_[:-1])
        elif list_[-1] == '↑':
            temp.append('+' + list_[:-1])
        else:
            temp.append('0')
    return temp

# 返回年月数据
def make_ym(lists):
    temp_y = []
    temp_m = []
    for list_ in lists:
        temp_y.append(list_[:4])
        temp_m.append(list_[5:7])
    return temp_y,temp_m

# 返回省市数据
def make_pro_city(province, new_city, year, i, j):
    temp_pro = []
    temp_city = []
    for k in range(len(year)):
        temp_pro.append(province[i])
        temp_city.append(new_city[i][j])
    return temp_pro,temp_city

# 返回房价数据
def make_dis(district,i,j,z,year):
    temp = []
    for k in range(len(year)):
        temp.append(district[i][j][z])
    return temp

# 返回链接数据
def make_link(links,i,j,z,year):
    temp = []
    for k in range(len(year)):
        temp.append(links)
    return temp

# 返回区县每年的链接用以访问
def make_each_year(link):
    dis = requests.get(link,headers=HEADERS)
    page = etree.HTML(dis.text)
    link_each_year = page.xpath('//*[starts-with(@class,"fjlist-box")]/h3/a/@href')
    return link_each_year

# 爬取县区的数据

In [35]:
import csv
import sys

# 将省中的'一线城市'改为'直辖市'
province[0] = '直辖市'

# 爬取中出现过需要验证的情况，所以需要用添加模式写入csv，并且在循环中加入判断进行接力赛
csv_link='C:/Users/钧娃/Desktop/result_v1.csv'
# 这里得用'a+'打开，不然就覆盖掉了
f = open(csv_link,'a+',encoding='GBK',newline='')
writer = csv.writer(f)
writer.writerow(['year','month','房价','涨跌幅','省','市','县区','链接'])

for i in range(0,len(district)):
    for j in range(len(district[i])):
#         if i == 21 and j < 10:
#             continue
        for z in range(len(district[i][j])):
#             if i == 21 and j == 10 and z < 2:
#                 continue
            link_each_year = make_each_year(district_links[i][j][z])
            for links in link_each_year: 
                dis = requests.get(links,headers=HEADERS)
                page = etree.HTML(dis.text)
                fd= page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/em/text()')
                ym = page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/b/text()')
                fj = page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/span/text()')
                fd = make_fd(fd)
                fj = make_fj(fj)
                year,month = make_ym(ym)
                pro,cit = make_pro_city(province, city, ym, i, j)
                dis = make_dis(district,i,j,z,ym)
                links = make_link(links,i,j,z,ym)
                rows = zip(year,month,fj,fd,pro,cit,dis,links)
                for row in rows:
                    writer.writerow(row)
#                 time.sleep()
f.close()

In [36]:
# 通过这个判断程序被终止在那个位置，用于继续爬取而不用从头开始，然后重新设置上面代码接着运行即可
for i in range(len(district)):
    for j in range(len(district[i])):
        for z in range(len(district[i][j])):
            if district_links[i][j][z] == 'http://www.anjuke.com/fangjia/xinzhou/yuanpingb/':
                print(i,j,z)

21 10 2


# 进行转换将csv转为要求的xlsx

In [37]:
# 获得了csv文件，然后转化为xlsx，直接用老师代码的话，会出现如下错误，excel限制表格中的链接不超过65,530个
import pandas as pd
df = pd.read_csv('F:/result.csv', encoding='GBK')
df.to_excel('F:/result.xlsx', encoding='GBK')

D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/xinganmeng2019/keerqinyouyizhongqi/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/xinganmeng2019/keerqinyouyiqianqi/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/xinganmeng2018/keerqinyouyiqianqi/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/xinganmeng2017/keerqinyouyiqianqi/' since it exceeds Excel's limit of 65,530 URLS per

D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/linyi2017/lanshana/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/linyi2016/lanshana/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/linyi2015/lanshana/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/linyi2019/luozhuangb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(

D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/dazhou2017/xuanhanb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/dazhou2016/xuanhanb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/dazhou2015/xuanhanb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/dazhou2019/kaijiangb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unico

D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/huzhou2019/changxingb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/huzhou2018/changxingb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/huzhou2017/changxingb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
D:\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:840: UserWarning: Ignoring URL 'http://www.anjuke.com/fangjia/huzhou2016/changxingb/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % forc

In [38]:
# 所以用另一种方法，建立一个writer，规定将string转化为url为否，即不转化
writer = pd.ExcelWriter(r'F:/result.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer)
writer.close()

In [2]:
# 完结保存即可，最终我们爬取了92030行数据，其实还可以加个几百行，因为没有爬取上海北京这些城市的平均房价。

# 我们组小伙伴发现没有海南省但是有海口市，最后单独补上就可以了

In [8]:
import requests
from lxml import etree
import csv
import sys

# 收集海口的县区
import time
haikou_link = 'https://www.anjuke.com/fangjia/haikou/'
temp = []
temp_links = []
district_response = requests.get(haikou_link,headers=HEADERS)
if etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()') != []:
    temp=(etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()'))
    temp_links=(etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/@href'))

In [10]:
temp_links

['http://www.anjuke.com/fangjia/haikou/meilan/',
 'http://www.anjuke.com/fangjia/haikou/longhuab/',
 'http://www.anjuke.com/fangjia/haikou/xiuying/',
 'http://www.anjuke.com/fangjia/haikou/qiongshan/',
 'http://www.anjuke.com/fangjia/haikou/chengmaixiana/',
 'http://www.anjuke.com/fangjia/haikou/lingaoxian/',
 'http://www.anjuke.com/fangjia/haikou/wenchangshia/',
 'http://www.anjuke.com/fangjia/haikou/danzhoushia/',
 'http://www.anjuke.com/fangjia/haikou/qionghaishi/',
 'http://www.anjuke.com/fangjia/haikou/haikouzhoubian/',
 'http://www.anjuke.com/fangjia/haikou/qitaby/']

In [18]:
csv_link='C:/Users/钧娃/Desktop/result.csv'
# 这里得用'a+'打开，不然就覆盖掉了
f = open(csv_link,'a+',encoding='GBK',newline='')
writer = csv.writer(f)
# writer.writerow(['year','month','房价','涨跌幅','省','市','县区','链接'])
pro = []
cit = []

for i in range(len(temp)):
    pro.append('海南')
    cit.append('海口')

for i in range(len(temp)):
    link_each_year = make_each_year(temp_links[i])
    for links in link_each_year: 
        dis = requests.get(links,headers=HEADERS)
        page = etree.HTML(dis.text)
        fd= page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/em/text()')
        ym = page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/b/text()')
        fj = page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/span/text()')
        fd = make_fd(fd)
        fj = make_fj(fj)
        year,month = make_ym(ym)
        dis = []
        links = []
        for ii in range(len(ym)):
            dis.append(temp[i])
            links.append(temp_links[i])
        rows = zip(year,month,fj,fd,pro,cit,dis,links)
        for row in rows:
            writer.writerow(row)
f.close()

In [20]:
import pandas as pd
df = pd.read_csv('F:/result.csv', encoding='GBK')
writer = pd.ExcelWriter(r'F:/result.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer)
writer.close()

# 又补充了三亚的数据

In [3]:
import requests
from lxml import etree
import csv
import sys

# 收集海口的县区
import time
haikou_link = 'https://www.anjuke.com/fangjia/sanya/'
temp = []
temp_links = []
district_response = requests.get(haikou_link,headers=HEADERS)
if etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()') != []:
    temp=(etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/text()'))
    temp_links=(etree.HTML(district_response.text).xpath('/html/body/div[2]/div[2]/div/span[2]/a/@href'))

In [4]:
temp_links

['http://www.anjuke.com/fangjia/sanya/hedonga/',
 'http://www.anjuke.com/fangjia/sanya/sanyawan/',
 'http://www.anjuke.com/fangjia/sanya/lingshuilub/',
 'http://www.anjuke.com/fangjia/sanya/dadonghai/',
 'http://www.anjuke.com/fangjia/sanya/haitangwan/',
 'http://www.anjuke.com/fangjia/sanya/wanning/',
 'http://www.anjuke.com/fangjia/sanya/hexiangxj/',
 'http://www.anjuke.com/fangjia/sanya/yacheng/',
 'http://www.anjuke.com/fangjia/sanya/yalongwan/',
 'http://www.anjuke.com/fangjia/sanya/xiaodonghai/',
 'http://www.anjuke.com/fangjia/sanya/tainyaqu/',
 'http://www.anjuke.com/fangjia/sanya/haitangqu/',
 'http://www.anjuke.com/fangjia/sanya/fenghuangdao/',
 'http://www.anjuke.com/fangjia/sanya/jiyangb/',
 'http://www.anjuke.com/fangjia/sanya/yanzhouqu/',
 'http://www.anjuke.com/fangjia/sanya/sanyazhoubain/',
 'http://www.anjuke.com/fangjia/sanya/qitaaaa/']

In [7]:
csv_link='C:/Users/钧娃/Desktop/result.csv'
# 这里得用'a+'打开，不然就覆盖掉了
f = open(csv_link,'a+',encoding='GBK',newline='')
writer = csv.writer(f)
# writer.writerow(['year','month','房价','涨跌幅','省','市','县区','链接'])
pro = []
cit = []

for i in range(len(temp)):
    pro.append('海南')
    cit.append('三亚')

for i in range(len(temp)):
    link_each_year = make_each_year(temp_links[i])
    for links in link_each_year: 
        dis = requests.get(links,headers=HEADERS)
        page = etree.HTML(dis.text)
        fd= page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/em/text()')
        ym = page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/b/text()')
        fj = page.xpath('//*[starts-with(@class,"fjlist-box")][1]/ul/li/a/span/text()')
        fd = make_fd(fd)
        fj = make_fj(fj)
        year,month = make_ym(ym)
        dis = []
        links = []
        for ii in range(len(ym)):
            dis.append(temp[i])
            links.append(temp_links[i])
        rows = zip(year,month,fj,fd,pro,cit,dis,links)
        for row in rows:
            writer.writerow(row)
f.close()

In [8]:
import pandas as pd
df = pd.read_csv('F:/result.csv', encoding='GBK')
writer = pd.ExcelWriter(r'F:/result.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer)
writer.close()

# 最终我们爬取了94018行数据